refer to this for installation https://github.com/abetlen/llama-cpp-python#installation-with-openblas--cublas--clblast--metal 

In [6]:
%pip install llama_index
%pip install llama-cpp-python
%pip install transformers
%pip install torch
%pip install tqdm



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [7]:
import logging
import sys

from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)  # Change INFO to DEBUG if you want more extensive logging
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

llm = LlamaCPP(
    model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf",
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 4}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [8]:
# Create an index of your documents
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

storage_directory = "./storage"

documents = SimpleDirectoryReader('./testdata').load_data()

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)


index = VectorStoreIndex.from_documents(documents, service_context=service_context)
# Persist the index to disk
index.storage_context.persist(persist_dir=storage_directory)

**********
Trace: index_construction
    |_node_parsing ->  0.033455 seconds
      |_chunking ->  0.005499 seconds
      |_chunking ->  0.006981 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.000998 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.000997 seconds
    |_embedding ->  0.249234 seconds
**********


In [9]:
# Now you can load the index from disk when needed, and not rebuild it each time.
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import StorageContext, load_index_from_storage

# transcript_directory = "transcripts/ancient-aliens-official"
storage_directory = "./storage"

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)

storage_context = StorageContext.from_defaults(persist_dir=storage_directory)
index = load_index_from_storage(storage_context, service_context=service_context)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
**********
Trace: index_construction
**********


In [11]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

# response = query_engine.query("What do you think of Facebook's LLaMa?")
# print(response)

# query = "What is the Alien nuclear agenda? \n\n" \
#         "Please summarize the information into 3 detailed paragraphs, something suitable for a blog post."

query = "could you combine the learning styles from the document and generate a few game types based on the octalysis framework for different types of games"

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit


**********
Trace: query
    |_query ->  384.138476 seconds
      |_retrieve ->  0.36931 seconds
        |_embedding ->  0.332298 seconds
      |_synthesize ->  383.766176 seconds
        |_templating ->  0.0 seconds
        |_llm ->  383.70524 seconds
**********


<b>  Based on the provided context information and the Octalysis framework, here are some game ideas that combine different learning styles and incorporate the eight core drives:

1. Virtual Reality Learning Adventure: This game combines visual, auditory, and kinesthetic learning styles. Players embark on a virtual reality adventure where they explore different environments, interact with objects, and complete challenges to earn rewards. The game incorporates the core drives of Epic Meaning and Calling (CD1), Development and Accomplishment (CD2), and Social Elevation (CD8).
2. Interactive Puzzle Game: This game combines logical and visual learning styles. Players solve puzzles and complete challenges to progress through the game. The game incorporates the core drives of Epic Meaning and Calling (CD1), Development and Accomplishment (CD2), and Curiosity and Exploration (CD4).
3. Multiplayer Debate Game: This game combines auditory and social learning styles. Players engage in debates on various topics, with the goal of convincing their opponents of their perspective. The game incorporates the core drives of Social Elevation (CD8), Development and Accomplishment (CD2), and Epic Meaning and Calling (CD1).
4. Virtual Lab Simulation: This game combines kinesthetic and logical learning styles. Players conduct experiments in a virtual lab, manipulating variables and observing the results. The game incorporates the core drives of Development and Accomplishment (CD2), Curiosity and Exploration (CD4), and Epic Meaning and Calling (CD1).
5. Story-Based Quiz Game: This game combines reading/writing and auditory learning styles. Players progress through a story, completing quizzes and challenges to unlock new content. The game incorporates the core drives of Development and Accomplishment (CD2), Curiosity and Exploration (CD4), and Social Elevation (CD8).
6. Virtual Scavenger Hunt: This game combines kinesthetic and naturalistic learning styles. Players embark on a virtual scavenger hunt, exploring different environments and completing challenges to earn rewards. The game incorporates the core drives of Development and Accomplishment (CD2), Curiosity and Exploration (CD4), and Epic Meaning and Calling (CD1).
7. Role-Playing Debate Game: This game combines auditory and social learning styles. Players engage in debates on various topics, with the goal of convincing their opponents of their perspective. The game incorporates the core drives of Social Elevation (CD8), Development and Accomplishment (CD2), and Epic Meaning and Calling (CD1).
8. Interactive Fitness Game: This game combines kinesthetic and naturalistic learning styles. Players engage in physical activities, such as exercise or sports, while completing challenges and earning rewards. The game incorporates the core drives of Development and Accomplishment (CD2), Curiosity and Exploration (CD4), and Social Elevation (CD8).

These game ideas combine different learning styles and incorporate the eight core drives of the Octalysis framework to create engaging and motivating experiences for learners.</b>

In [ ]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

# response = query_engine.query("What do you think of Facebook's LLaMa?")
# print(response)

# query = "What is the Alien nuclear agenda? \n\n" \
#         "Please summarize the information into 3 detailed paragraphs, something suitable for a blog post."

query = "I am trying to develop a game to enhance adult learning learning \n\n" \
        "Given that the game is a narrative driven adventure game, please generate a question regarding sexual harrassment \n\n"

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))